**Functional redundancy and resilience**

Examination of functional redundancy and reslience in the context of microbial biodiversity measurements.
1. compare taxonomic and functional diversity in these same samples to evaluate _functional redundancy_ in microbial communities.
2. examine longitudinal change in these biodiversity metrics to measure _resilience_ in microbial communities

- Metadata : sample_meta_data (done in A)
- Diversity analysis : alpha = core-metrics-results (done in F), beta = core-metrics-results-bd (done in G)
- Metagenome content predicted by PICRUST2 : (done in I)


<a id='setup'></a>
## 0. Setup

In [6]:
import os
import qiime2 as q2
import pandas as pd
from qiime2 import Visualization


    
# do not increase this value!
n_jobs = 3
    
%matplotlib inline

In [7]:
data_dir ='project_data'

In [11]:
md = q2.Metadata.load(data_dir + '/sample_meta_data.tsv').to_dataframe()
pd.DataFrame([str(sorted(md[col].astype(str).unique())) for col in md.columns],
             index=pd.Index(md.columns, name='Column'), columns=['Values'])

,Values
Column,
GEN_age_cat,"['20s', '30s', '40s', '50s', '60s', '70+', 'No..."
GEN_age_corrected,"['11.0', '14.0', '15.0', '16.0', '17.0', '18.0..."
GEN_bmi_cat,"['Normal', 'Not provided', 'Obese', 'Overweigh..."
GEN_bmi_corrected,"['11.2', '11.57', '14.34', '14.62', '14.79', '..."
GEN_cat,"['False', 'True']"
GEN_collection_timestamp,"['2014-05-10 12:00:00', '2015-01-01 09:00:00',..."
GEN_country,"['Australia', 'Belgium', 'Canada', 'Georgia', ..."
GEN_dog,"['False', 'True']"
GEN_elevation,"['-0.9', '0.0', '1.4', '10.4', '10.8', '10.9',..."


<a id='function'></a>
## 1.a Functional redundancy calculation

Next we will look at predicted gene pathway information to compare taxonomic vs. functional diversity patterns. 
We will use the `core-metrics` pipeline on the `pathway_abundance.qza` table, which consists of PICRUST2-predicted gene pathway counts. 

In [5]:
! qiime diversity core-metrics \
  --i-table $data_dir/pathway_abundance.qza \
  --m-metadata-file $data_dir/sample_meta_data.tsv \
  --p-sampling-depth 100000 \
  --p-n-jobs $n_jobs \
  --output-dir $data_dir/core-metrics-picrust2  

Saved FeatureTable[Frequency] to: project_data/core-metrics-picrust2/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: project_data/core-metrics-picrust2/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: project_data/core-metrics-picrust2/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: project_data/core-metrics-picrust2/evenness_vector.qza
Saved DistanceMatrix to: project_data/core-metrics-picrust2/jaccard_distance_matrix.qza
Saved DistanceMatrix to: project_data/core-metrics-picrust2/bray_curtis_distance_matrix.qza
Saved PCoAResults to: project_data/core-metrics-picrust2/jaccard_pcoa_results.qza
Saved PCoAResults to: project_data/core-metrics-picrust2/bray_curtis_pcoa_results.qza
Saved Visualization to: project_data/core-metrics-picrust2/jaccard_emperor.qzv
Saved Visualization to: project_data/core-metrics-picrust2/bray_curtis_emperor.qzv


## Hypothesis 1 : Milk products

In [9]:
Visualization.load(f'{data_dir}/core-metrics-picrust2/adonis-bc-h1.qzv')

<visualization: Visualization uuid: 2a97939f-3e37-497f-b67b-d48edc19ca94>

In [8]:
! qiime diversity adonis \
  --i-distance-matrix $data_dir/core-metrics-picrust2/bray_curtis_distance_matrix.qza \
  --m-metadata-file $data_dir/sample_meta_data.tsv \
  --p-formula "NUT_milk_cheese_frequency*NUT_milk_substitute_frequency*NUT_vitamin_d_supplement_frequency" \
--o-visualization $data_dir/core-metrics-picrust2/adonis-bc-h1.qzv

Saved Visualization to: project_data/core-metrics-picrust2/adonis-bc-h1.qzv


## Hypothesis 2 : Mediterranean

In [11]:
! qiime diversity adonis \
  --i-distance-matrix $data_dir/core-metrics-picrust2/bray_curtis_distance_matrix.qza \
  --m-metadata-file $data_dir/sample_meta_data.tsv \
  --p-formula "NUT_olive_oil*NUT_seafood_frequency*NUT_vegetable_frequency*NUT_fruit_frequency*NUT_whole_grain_frequency" \
--o-visualization $data_dir/core-metrics-picrust2/adonis-bc-h2.qzv

Saved Visualization to: project_data/core-metrics-picrust2/adonis-bc-h2.qzv


In [12]:
Visualization.load(f'{data_dir}/core-metrics-picrust2/adonis-bc-h2.qzv')

<visualization: Visualization uuid: 58099508-3d53-4ec1-b2e5-2cf64fbddb25>

## Hypothesis 3 : Poultry

In [13]:
! qiime diversity adonis \
  --i-distance-matrix $data_dir/core-metrics-picrust2/bray_curtis_distance_matrix.qza \
  --m-metadata-file $data_dir/sample_meta_data.tsv \
  --p-formula "NUT_whole_eggs*NUT_poultry_frequency" \
--o-visualization $data_dir/core-metrics-picrust2/adonis-bc-h3.qzv

Saved Visualization to: project_data/core-metrics-picrust2/adonis-bc-h3.qzv


In [14]:
Visualization.load(f'{data_dir}/core-metrics-picrust2/adonis-bc-h3.qzv')

<visualization: Visualization uuid: 20972341-27cf-45c9-bfc8-a6ac47e1f0df>

## Hypothesis 4 : Meat

In [15]:
! qiime diversity adonis \
  --i-distance-matrix $data_dir/core-metrics-picrust2/bray_curtis_distance_matrix.qza \
  --m-metadata-file $data_dir/sample_meta_data.tsv \
  --p-formula "NUT_high_fat_red_meat_frequency*NUT_meat_eggs_frequency*NUT_red_meat_frequency*NUT_vitamin_b_supplement_frequency" \
--o-visualization $data_dir/core-metrics-picrust2/adonis-bc-h4.qzv

Saved Visualization to: project_data/core-metrics-picrust2/adonis-bc-h4.qzv


In [16]:
Visualization.load(f'{data_dir}/core-metrics-picrust2/adonis-bc-h4.qzv')

<visualization: Visualization uuid: d11b9012-6396-4187-adcd-c9f0e15c7b2e>

## Hypothesis 5 : Drinks

In [17]:
! qiime diversity adonis \
  --i-distance-matrix $data_dir/core-metrics-picrust2/bray_curtis_distance_matrix.qza \
  --m-metadata-file $data_dir/sample_meta_data.tsv \
  --p-formula "NUT_alcohol_frequency*NUT_drinks_per_session" \
--o-visualization $data_dir/core-metrics-picrust2/adonis-bc-h5.qzv

Saved Visualization to: project_data/core-metrics-picrust2/adonis-bc-h5.qzv


In [ ]:
Visualization.load(f'{data_dir}/core-metrics-picrust2/adonis-bc-h5.qzv')

<a id='procrustes'></a>
## 1.b Comparing ordinations

One way to compare beta diversity ordination results directly is with [Procrustes analysis](https://en.wikipedia.org/wiki/Procrustes_analysis). This method rotates and scales two ordinations to align them as best as possible. We can view the transformed PCoA coordinates together in a single plot to visually compare the ordinations. 



In [14]:
! qiime diversity procrustes-analysis \
  --i-reference $data_dir/core-metrics-results/bray_curtis_pcoa_results.qza \
  --i-other $data_dir/core-metrics-picrust2/bray_curtis_pcoa_results.qza \
  --output-dir $data_dir/core-metrics-picrust2/procrustes/

! qiime emperor procrustes-plot \
  --i-reference-pcoa $data_dir/core-metrics-picrust2/procrustes/transformed_reference.qza \
  --i-other-pcoa $data_dir/core-metrics-picrust2/procrustes/transformed_other.qza \
  --m-metadata-file $data_dir/sample_meta_data.tsv \
  --o-visualization $data_dir/core-metrics-picrust2/procrustes-pcoa-plot.qzv

Plugin error from diversity:

  The matrices cannot be fitted unless they have the same dimensions

Debug info has been saved to /tmp/qiime2-q2cli-err-p_re2oxl.log
Usage: qiime emperor procrustes-plot [OPTIONS]

  Plot two procrustes-fitted matrices

Inputs:
  --i-reference-pcoa ARTIFACT
    PCoAResults          The reference ordination matrix to be plotted.
                                                                    [required]
  --i-other-pcoa ARTIFACT
    PCoAResults          The "other" ordination matrix to be plotted (the one
                         that was fitted to the reference).         [required]
  --i-m2-stats ARTIFACT ProcrustesStatistics
                         The M^2 value of the procrustes analysis & its
                         associated p value.                        [optional]
Parameters:
  --m-metadata-file METADATA...
    (multiple            The sample metadata.
     arguments will be   
     merged)                                                     

In [ ]:
Visualization.load(f'{data_dir}/core-metrics-picrust2/procrustes-pcoa-plot.qzv')

#here we say how good does this fit look 
#does this indicates that ASV and pathway abundances are very similar or dissimilar